In [33]:
%matplotlib inline
from __future__ import division
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u
from astropy.table import Table, hstack, vstack, join
from scipy.stats import ks_2samp, beta
from functools import reduce

import math
import mpl_style
import csv
import hex_scatter as hs

H0 = 73.
cosmo = FlatLambdaCDM(H0, Om0=0.3)
h = H0/100

In [2]:
alldata = Table(fits.open('../JHU_mstell_ssfr_Yang_gzoo.fits')[1].data)
alldata['col10'].name = 'color' #^{0.1}(g-r), K-E corrected to z=0.1, petro
alldata['col9'].name = 'M_r-5logh' #^{0.1}(M_r) - 5\log h, K-E corrected to z=0.1, petro
alldata['col4_2'].name = 'centsat' #if BRIGHTEST galaxy in group, is 1, otherwise 2
alldata['t01_smooth_or_features_a02_features_or_disk_weighted_fraction'].name = 'pfeatures'
alldata['AVG_SSFR'].name = 'ssfr'
alldata['Z'].name = 'z'
alldata['Z_WARNING'].name = 'z_warn'
alldata['AVG_MSTELL'].name = 'mstell'
alldata['col3_2'].name = 'group_num'
alldata['col1_2'].name = 'galaxy_num'
alldata['col2_1'].name = 'NYU-VAGC_ID'

hdulist2 = fits.open('../petroC_group.fits')
groupdata = Table(hdulist2[1].data)
groupdata['col1'].name = 'group_num'
groupdata['col7'].name = 'halo_mass_lum'#Luminosity weighted halo mass estimate, units: log M_halo/ (M_{\odot}/h)
groupdata['col8'].name = 'halo_mass_mstell'#Mstellar weighted halo mass estimate, units: log M_halo/ (M_{\odot}/h)
hdulist2.close()

In [3]:
#Magnitude limit calculation
z_lim = 0.06
d = cosmo.luminosity_distance(z_lim).to(u.pc)
m = 17
M_limit = m - 5*math.log(d.value,10) + 5 
adj_M_limit = M_limit - 5*math.log(h,10)
print(M_limit, adj_M_limit)

(-20.05490117819621, -19.37151547879849)


In [4]:
#turn alldata into galaxy table
#cuts to make - magnitude, redshift
#things to add - halo mass, red/blue, sfing/passive
    #then go to group table 
    #also make group_list

halo_mass_lum_gal = []
halo_mass_mstell_gal = []
redblue_gal = []
pasSFing_gal = []

group_list = [[67926]]

for i in np.arange(len(alldata)):
    group_ind = np.where(groupdata['group_num'] == alldata['group_num'][i])
    
    halo_mass_lum_gal.append(groupdata['halo_mass_lum'][group_ind[0]][0] - np.log10(h)) 
    halo_mass_mstell_gal.append(groupdata['halo_mass_mstell'][group_ind[0]][0] - np.log10(h))
    
    color_line = 0.7 - 0.032*(alldata['M_r-5logh'][i] + 16.5) #formula from Weinmann et al 2006, includes -5log(h)
    active_line = -10 + 0.094*(alldata['M_r-5logh'][i] + 15)
    
    if alldata['ssfr'][i] <= active_line:
        pasSFing_gal.append('passive')
    elif alldata['ssfr'][i] > active_line:
        pasSFing_gal.append('SFing')
    
    if alldata['color'][i] > color_line:
        redblue_gal.append('red')
    elif alldata['color'][i] <= color_line:
        redblue_gal.append('blue')
            
alldata['halo_mass_lum'] = halo_mass_lum_gal
alldata['halo_mass_mstell'] = halo_mass_mstell_gal
alldata['redorblue'] = redblue_gal
alldata['passiveorSFing'] = pasSFing_gal

z_cut_ind = np.intersect1d(np.where(alldata['z'] > 0.01)[0], np.where(alldata['z'] < 0.06)[0])
mag_cut_ind = np.where(alldata['M_r-5logh'] < adj_M_limit)
cut_ind = np.intersect1d(z_cut_ind, mag_cut_ind)

cutdata = alldata[cut_ind]
galaxy_table = Table([cutdata['galaxy_num'], cutdata['group_num'], cutdata['NYU-VAGC_ID'], cutdata['z'], cutdata['mstell'], cutdata['ssfr'], cutdata['pfeatures'], cutdata['centsat'], cutdata['M_r-5logh'], cutdata['color'], cutdata['halo_mass_lum'], cutdata['halo_mass_mstell'], cutdata['redorblue'], cutdata['passiveorSFing']])

In [26]:
#galaxy_table = Table([cutdata['galaxy_num'], cutdata['group_num'], cutdata['NYU-VAGC_ID'], cutdata['z'], cutdata['mstell'], cutdata['ssfr'], cutdata['pfeatures'], cutdata['centsat'], cutdata['M_r-5logh'], cutdata['color'], cutdata['halo_mass_lum'], cutdata['halo_mass_mstell'], cutdata['redorblue'], cutdata['passiveorSFing']])
#print(len(galaxy_table))
#print(len(group_list))
#print(len(set(galaxy_table['group_num'])))

#print(np.where(galaxy_table['group_num'] == 2419))
galaxy_table.info

<Table length=42919>
      name        dtype 
---------------- -------
      galaxy_num   int64
       group_num   int64
     NYU-VAGC_ID   int64
               z float32
          mstell float32
            ssfr float32
       pfeatures float32
         centsat   int64
       M_r-5logh float64
           color float64
   halo_mass_lum float64
halo_mass_mstell float64
       redorblue    str4
  passiveorSFing    str7

In [6]:
#creates group table
group_nums = []
cent_ssfr = []
cent_sfing = []
cent_pfeat = []
cent_mr = []
cent_mass = []
halo_mass_lum_group = []
halo_mass_mstell_group = []
sf_frac = []
spiral_frac = []
smooth_frac = []
blue_frac = []
median_sat_pfeat = []
spiral_half_frac = []
gal_num = []

cent_redblue = []

for group in set(galaxy_table['group_num']): #loop thru groups
    group_gals_ind = np.where(galaxy_table['group_num'] == group)[0] #indices of galaxies in a given group
    
    if len(group_gals_ind) > 1: #remove single galaxy groups
        has_central = False 
        sat_pfeat = []
        sat_spiral = 0
        sat_smooth = 0
        sat_sfing = 0
        sat_spiral_half = 0
        sat_blue = 0
        sat_num = 0
        
        for gal_ind in group_gals_ind: #now loop through these galaxies
            if galaxy_table['centsat'][gal_ind] == 1: #if its a central galaxy
                cent_ssfr.append(galaxy_table['ssfr'][gal_ind])
                cent_pfeat.append(galaxy_table['pfeatures'][gal_ind])
                cent_mr.append(galaxy_table['M_r-5logh'][gal_ind])
                halo_mass_lum_group.append(galaxy_table['halo_mass_lum'][gal_ind])
                halo_mass_mstell_group.append(galaxy_table['halo_mass_mstell'][gal_ind])
                cent_mass.append(galaxy_table['mstell'][gal_ind])
                cent_redblue.append(galaxy_table['redorblue'][gal_ind])
                cent_sfing.append(galaxy_table['passiveorSFing'][gal_ind])
                has_central = True
            else: #satellite galaxy
                sat_pfeat.append(galaxy_table['pfeatures'])
                sat_num += 1
                if galaxy_table['pfeatures'][gal_ind] >= 0.8: #featured
                    sat_spiral += 1
                if galaxy_table['pfeatures'][gal_ind] <= 0.2: #smooth
                    sat_smooth += 1
                if galaxy_table['passiveorSFing'][gal_ind] == 'SFing': #star forming 
                    sat_sfing += 1
                if galaxy_table['pfeatures'][gal_ind] >= 0.5: #featured by alt defn
                    sat_spiral_half += 1
                if galaxy_table['redorblue'][gal_ind] == 'blue':
                    sat_blue += 1

        if has_central == True: #only add group to table if it has a central
            median_sat_pfeat.append(np.median(sat_pfeat))
            sf_frac.append(sat_sfing/sat_num)
            spiral_frac.append(sat_spiral/sat_num)
            spiral_half_frac.append(sat_spiral_half/sat_num)
            smooth_frac.append(sat_smooth/sat_num)
            blue_frac.append(sat_blue/sat_num)
            gal_num.append(sat_num+1)
            group_nums.append(group)
            
group_table = Table([group_nums, gal_num, halo_mass_lum_group, halo_mass_mstell_group, cent_ssfr, cent_sfing, cent_redblue, cent_pfeat, cent_mass, sf_frac, blue_frac, spiral_frac, smooth_frac, spiral_half_frac, median_sat_pfeat],
                    names=('group_num', 'n_gal', 'halo_mass_lum', 'halo_mass_mstell', 'cent_ssfr', 'sfing_central', 'cent_redblue', 'cent_pfeat', 'cent_mstell', 'sat_sf_frac', 'sat_blue_frac', 'sat_spiral_frac', 'sat_smooth_frac', 'sat_spiral_half_frac', 'median_sat_pfeat'))


In [37]:
#add group data to individual galaxies in galaxy_data

n_gal = []
cent_ssfr = []
SFing_central = []
central_pfeat = []
cent_mass = []
sat_SF_frac = []
sat_spiral_frac = []
sat_smooth_frac = []
sat_blue_frac = []
cent_redblue = []
median_sat_pfeat = []
sat_spiral_half_frac = []
galaxy_num_add = []


single_gal_groups = 0

for g in range(len(galaxy_table)): #loop thru galaxies
    ind = np.where(group_table['group_num'] == galaxy_table['group_num'][g])[0]
    if ind.size > 0: #if the group of galaxy g is in the group table
        galaxy_num_add.append(galaxy_table['galaxy_num'].data[g])
        
        n_gal.append(group_table['n_gal'].data[ind][0])
        cent_ssfr.append(group_table['cent_ssfr'].data[ind][0])
        SFing_central.append(group_table['sfing_central'].data[ind][0])
        central_pfeat.append(group_table['cent_pfeat'].data[ind][0])
        cent_mass.append(group_table['cent_mstell'].data[ind][0])
        sat_SF_frac.append(group_table['sat_sf_frac'].data[ind][0])
        sat_spiral_frac.append(group_table['sat_spiral_frac'].data[ind][0])
        sat_smooth_frac.append(group_table['sat_smooth_frac'].data[ind][0])
        sat_blue_frac.append(group_table['sat_blue_frac'].data[ind][0])
        cent_redblue.append(group_table['cent_redblue'].data[ind][0])
        median_sat_pfeat.append(group_table['median_sat_pfeat'].data[ind][0])
        sat_spiral_half_frac.append(group_table['sat_spiral_half_frac'].data[ind][0])
    else:
        single_gal_groups += 1
group_data_add = Table([galaxy_num_add, n_gal, cent_ssfr, cent_redblue, SFing_central, central_pfeat, cent_mass, sat_SF_frac, sat_blue_frac, sat_spiral_frac, sat_smooth_frac, sat_spiral_half_frac, median_sat_pfeat],
                                   names=('galaxy_num', 'n_gal', 'cent_ssfr', 'cent_redblue', 'sfing_central', 'cent_pfeat', 'cent_mass', 'sat_sf_frac', 'sat_blue_frac', 'sat_spiral_frac', 'sat_smooth_frac', 'sat_spiral_half_frac', 'median_sat_pfeat'))
                                   #dtype=('i4', 'f8', 'S8', 'S8', 'f8', 'f8', 'f8', 'f8', 'f8'))

    
#the issue is this hstack here - should be a match based on galaxy number
galaxy_table_joined = join(galaxy_table, group_data_add, keys='galaxy_num', join_type='inner')
print(len(galaxy_table))

42919


In [42]:
print(len(group_data_add))
print(len(galaxy_table))
#print(np.where(group_data_add['galaxy_num']==2419))
print(np.where(galaxy_table['group_num']==2419))
galaxy_table2 = hstack([galaxy_table, group_data_add], join_type='inner')
print(len(galaxy_table2))
print(np.where(galaxy_table_joined['group_num']==2419))

12352
42919
(array([27753, 27754, 27755, 27756]),)
12352
(array([7768, 7820, 7826, 7827]),)


In [28]:
print(len(cutdata))
print(len(galaxy_table))
print(single_gal_groups)
print(len(galaxy_table) + single_gal_groups)

42919
42919
30567
73486


In [43]:
group_table.write("../group_table2.fits", format='fits', overwrite=True)
galaxy_table.write("../galaxy_table2.fits", format='fits', overwrite=True)